# AT Code Challenge

## Description
The task is to produce a rate-limiting module that stops a particular requestor from making too many http requests within a particular period of time.

The module should expose a method that keeps track of requests and limits it such that a requester can only make 100 requests per hour. After the limit has been reached, return a 429 with the text "Rate limit exceeded. Try again in #{n} seconds".

Although you are only required to implement the strategy described above, it should be easy to extend the rate limiting module to take on different rate-limiting strategies.

How you do this is up to you. Think about how easy your rate limiter will be to maintain and control. Write what you consider to be production-quality code, with comments and tests if and when you consider them necessary.

In [ ]:
# Inspirations

# Flask Response
# https://stackoverflow.com/questions/7824101/return-http-status-code-201-in-flask

# Decorators
# https://realpython.com/primer-on-python-decorators/

# Rate Limiter
# https://pypi.org/project/ratelimiter/

# Non Local
# https://stackoverflow.com/questions/5218895/python-nested-functions-variable-scoping

# Difference between datetime objects
# https://stackoverflow.com/questions/1345827/how-do-i-find-the-time-difference-between-two-datetime-objects-in-python

# Clearing the cache of the decorator
# https://stackoverflow.com/questions/58626079/how-to-clean-reset-cache-memory-of-my-decorator

## Rate Limiter

In [ ]:
from flask import Response
import datetime

In [ ]:
# Datetime helper
def deltaHours(timeA, timeB):
    duration = timeA - timeB
    duration = duration.total_seconds()
    hours = divmod(duration, 3600)[0]
    return hours

In [ ]:
def rateLimiter(func):
    latestCallTimes = {}
    
    def wrapper(obj, uuid, timeOfCall = datetime.datetime.now()):
        nonlocal latestCallTimes
        
        now = datetime.datetime.now()
        
        if uuid in latestCallTimes:
            latestCallTimes[uuid] = [time for time in latestCallTimes[uuid] if deltaHours(now, time) < 1.]
        else:
            latestCallTimes[uuid] = []
        
        if (len(latestCallTimes[uuid]) < 100):
            latestCallTimes[uuid].append(timeOfCall)
            return func(obj)
        else:
            duration = now - latestCallTimes[uuid][0]
            duration = duration.total_seconds()
            secondsLeft = 3600 - duration
            
            return Response("Rate limit exceeded. Try again in {0} seconds".format(secondsLeft), 
                            status=429, 
                            mimetype='application/json')
    
    wrapper.cache_reset = lambda : latestCallTimes.clear()
    
    return wrapper

## Tests

In [ ]:
import numpy as np
from enum import Enum
import uuid

In [ ]:
def printResponse(response):
    print(response.status, response.get_data())

In [ ]:
class TestResult(Enum):
    passed = 1
    failed = 2

class Test:    
    class Server():
        @rateLimiter
        def getResponse(self):
            return Response("success", status=200, mimetype='application/json')
    
    def setup(self):
        self.Server.getResponse.cache_reset()
    
    def run():
        return TestResult.failed

In [ ]:
class RateLimiterTest1(Test):
    def run(self):
        
        server = Test.Server()
        
        for i in range(1, 102):
            response = server.getResponse("")
            if (response.status_code == 429):
                break
            if (i > 100):
                return TestResult.failed
        
        return TestResult.passed

In [ ]:
class RateLimiterTest2(Test):
    def run(self):
        
        server = Test.Server()
        
        times = datetime.datetime.now() + np.linspace(-1., 0., 200) * datetime.timedelta(hours=0.5)
        
        responses = [server.getResponse("", time) for time in times]
        
        for response in responses[0:100]:
            if (response.status_code != 200):
                return TestResult.failed
         
        for response in responses[100:]:
            if (response.status_code != 429):
                return TestResult.failed
        
        return TestResult.passed

In [ ]:
class RateLimiterTest3(Test):
    def run(self):
        
        server = Test.Server()
        
        now = datetime.datetime.now()
        
        times = now + np.linspace(-1., 0., 100) * datetime.timedelta(hours=1.)
        
        responses = [server.getResponse("", time) for time in times]
        
        for response in responses:
            if (response.status_code != 200):
                return TestResult.failed
        
        return TestResult.passed

In [ ]:
class RateLimiterTest4(Test):
    def run(self):
        
        server = Test.Server()
        
        now = datetime.datetime.now()
        
        times = now + np.linspace(-1., 0., 200) * datetime.timedelta(hours=1.0)
        
        responses = [server.getResponse("", time) for time in times]
                
        for response in responses[0:100]:
            if (response.status_code != 200):
                return TestResult.failed
         
        for response in responses[101:]:
            if (response.status_code != 429):
                return TestResult.failed
        
        return TestResult.passed

In [ ]:
class RateLimiterTest5(Test):
    
    class Client():
        def __init__(self):
            self.uuid = uuid.uuid4()
    
    def run(self):
        server = self.Server()
        clients = [self.Client(), self.Client(), self.Client()]
        
        for i in range(1, 102):
            for client in clients:
                response = server.getResponse(client.uuid)
                
                if (i <= 100):
                    if response.status_code == 429:
                        return TestResult.failed
                else:
                    if response.status_code == 200:
                        return TestResult.failed
        
        return TestResult.passed

In [ ]:
tests = [RateLimiterTest1(), RateLimiterTest2(), RateLimiterTest3(), RateLimiterTest4(), RateLimiterTest5()]
results = []

for i, test in enumerate(tests):
    test.setup()
    result = test.run()
    print ("Test", i + 1, result)
    results.append(result)
    
passed = [result for result in results if result == TestResult.passed]

print ("Test Results:", len(passed), "/", len(tests), "tests passed")